In [1]:
import warnings
warnings.filterwarnings("ignore")

import argparse
import logging
import os
import sys
import time


from collections import defaultdict, OrderedDict

import matplotlib
import numpy as np
import soundfile as sf
import torch
import yaml

from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from tqdm import tqdm

import sys
sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from basic import plot_spectrogram
from sound import play_audio

sys.path.append("../")

from pyfiles.dataset import Parallelo2oVCMelDataset

import seq2seq_vc
import seq2seq_vc.models
import seq2seq_vc.losses
import seq2seq_vc.trainers
import seq2seq_vc.collaters

# from seq2seq_vc.datasets import ParallelVCMelDataset
from torch.utils.data import Dataset

from seq2seq_vc.utils import read_hdf5
from seq2seq_vc.utils.types import str_or_none

# set to avoid matplotlib error in CLI environment
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt

from seq2seq_vc.schedulers.warmup_lr import WarmupLR

scheduler_classes = dict(warmuplr=WarmupLR)

class Dict2Obj(object):
    def __init__(self, dictionary):
        """Constructor"""
        for key in dictionary:
            setattr(self, key, dictionary[key])

import joblib
import glob
datasplit = list(np.load("./data_split_ARCTIC.npy", allow_pickle=True))

In [2]:
from torch.utils.data import Dataset
class ParallelArcticDataset(Dataset):
    def __init__(self, src_dir, trg_dir, datasplit, scaler, mode="train", input_output=["mel", "wavlm"]):
        modefiles = datasplit[["train", "valid", "test"].index(mode)]
        filenames = [os.path.basename(a)[:-4] for a in glob.glob(src_dir+f"{input_output[0]}/*")]
        filenames.sort()
        files = []
        for fn in filenames:
            if fn in modefiles:
                exist = True
                if not(os.path.exists(f"{trg_dir}{input_output[1]}/{fn}.npy")):
                    exist = False
                    break
                if exist:
                    files += [fn]
        data = {}
        data["src"] = [src_dir + f"{input_output[0]}/{fn}.npy" for fn in files]
        data["trg"] = [trg_dir + f"{input_output[1]}/{fn}.npy" for fn in files]
            
        self.data = data
        self.scaler = scaler
        self.files = files
        self.input_output = input_output
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        items = {}
        src_mel = self.data["src"][idx]
        trg_mel = self.data["trg"][idx]
        items["src_feat"] = self.scaler[self.input_output[0]].transform(np.load(src_mel).T)
        items["trg_feat"] = self.scaler[self.input_output[1]].transform(np.load(trg_mel).T)
        items["src_condition"] = np.load(src_mel.replace("mel", "accent_embedding"))
        items["trg_condition"] = np.load(trg_mel.replace("mel", "accent_embedding"))
        items["utt_id"] = self.files[idx]
        items["accent_id"] = 0
        
        return items

In [3]:
# Dataset Variables
src_dir = "/mntcephfs/lab_data/shoinoue/Dataset/CMU-ARCTIC/SLT/"
trg_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/Hindi/"

scaler = {}
scaler_filename = f"ckpts/scalers/LibriTTS-R_16000.save"
scaler["mel"] = joblib.load(scaler_filename)
scaler_filename = f"ckpts/scalers/LibriTTS-R_wavlm.save"
scaler["wavlm"] = joblib.load(scaler_filename)

In [4]:
size = "small"
conditiontype = "nocondition"
inputoutput = ["mel", "mel"]

args = {}
args["rank"] = 0
args["outdir"] = f"/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/{conditiontype}_accentaddition_gt2syn_{''.join(inputoutput)}_{size}_cutsilence/"
args["config_path"] = f"./../egs/l2-arctic/cascade/conf/{size}m2mvtn.{''.join(inputoutput)}.yaml"
args["init_checkpoint"] = ""
args["resume"] = ""
args["distributed"] = False
args = Dict2Obj(args)

# load main config
with open(args.config_path) as f:
    config = yaml.load(f, Loader=yaml.Loader)
config.update(vars(args))

# Customization
config["batch_size"] = 64
config["model_params"]["conditiontype"] = conditiontype
config["optimizer_params"]["lr"] = 0.00008

In [5]:
device = torch.device("cuda")
torch.backends.cudnn.benchmark = True
torch.cuda.set_device(args.rank)
if not os.path.exists(args.outdir):
    os.makedirs(args.outdir)
    
### Dataset Preparation ###
dataset = {
    "train": ParallelArcticDataset(src_dir, trg_dir, datasplit, scaler, "train", input_output=inputoutput),
    "dev": ParallelArcticDataset(src_dir, trg_dir, datasplit, scaler, "valid", input_output=inputoutput),
}

collater_class = getattr(
    seq2seq_vc.collaters,
    config.get("collater_type", "ARM2MVCCollater"),
)
collater = collater_class()

sampler = {"train": None, "dev": None}
data_loader = {
    "train": DataLoader(
        dataset=dataset["train"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["train"],
        pin_memory=config["pin_memory"],
    ),
    "dev": DataLoader(
        dataset=dataset["dev"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["dev"],
        pin_memory=config["pin_memory"],
    ),
}

### Model Preparation ###
model_class = getattr(
    seq2seq_vc.models,
    config.get("model_type", "M2MVTN"),
)
model = model_class(**config["model_params"]).to(device)

if config.get("criterions", None):
    criterion = {
        criterion_class: getattr(seq2seq_vc.losses, criterion_class)(
            **criterion_paramaters
        )
        for criterion_class, criterion_paramaters in config["criterions"].items()
    }
else:
    raise ValueError("Please specify criterions in the config file.")

### optimizers and schedulers ###
optimizer_class = getattr(
    torch.optim,
    # keep compatibility
    config.get("optimizer_type", "Adam"),
)
optimizer = optimizer_class(
    model.parameters(),
    **config["optimizer_params"],
)
scheduler_class = scheduler_classes.get(config.get("scheduler_type", "warmuplr"))
scheduler = scheduler_class(
    optimizer=optimizer,
    **config["scheduler_params"],
)

### define trainer ###
trainer_class = getattr(
    seq2seq_vc.trainers,
    config.get("trainer_type", "ARM2MVCTrainer"),
)
trainer = trainer_class(
    steps=0,
    epochs=0,
    data_loader=data_loader,
    sampler=sampler,
    model=model,
    vocoder=None,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    config=config,
    device=device,
)

# load pretrained parameters from checkpoint
if len(args.init_checkpoint) != 0:
    trainer.load_trained_modules(
        args.init_checkpoint, init_mods=config["init-mods"]
    )

# resume from checkpoint
if len(args.resume) != 0:
    trainer.load_checkpoint(args.resume)

# freeze modules if necessary
if config.get("freeze-mods", None) is not None:
    assert type(config["freeze-mods"]) is list
    trainer.freeze_modules(config["freeze-mods"])

In [6]:
try:
    trainer.run()
finally:
    trainer.save_checkpoint(
        os.path.join(config["outdir"], f"checkpoint-{trainer.steps}steps.pkl")
    )
    logging.info(f"Successfully saved checkpoint @ {trainer.steps}steps.")

[train]:   0%|          | 8/500000 [00:09<107:17:48,  1.29it/s]
KeyboardInterrupt



In [ ]:
# from pyfiles.feature_extractor import get_vocos
# ### Vocoder Preparation ###
# data_dir = "/mntcephfs/lab_data/shoinoue/"
# # fs = 24000
# fs = 16000

# if fs==24000:
#     config_path = f"{data_dir}Models/trained_models/vocos/24k/config.yaml"
#     model_path = f"{data_dir}Models/trained_models/vocos/24k/pytorch_model.bin"
# elif fs==16000:
#     config_path = f"{data_dir}Models/trained_models/vocos/vocos16k_noncausal_tealab/config16k.yaml"
#     model_path = f"{data_dir}Models/trained_models/vocos/vocos16k_noncausal_tealab/vocos16k_noncausal_last.ckpt"
# vocoder = get_vocos(config_path, model_path, fs)

# mel = dataset[0]["src_feat"]
# mel = scaler["mel"].inverse_transform(mel)
# y = vocoder.decode(torch.tensor(mel.T).unsqueeze(0)).cpu().numpy()
# play_audio(y, fs)

# mel = dataset[0]["trg_feat"]
# mel = scaler["mel"].inverse_transform(mel)
# y = vocoder.decode(torch.tensor(mel.T).unsqueeze(0)).cpu().numpy()
# play_audio(y, fs)